In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/signate/medical_treatises/data/01'

PROB = True
# TH = 0.018
TH = 0.020

BORDER = 0.023282372444280715



```
sub_sugi_UMLSBert_ENG_fold4_epoch5.csv    0.9177989 
sub_sugi_BiomedNLP-PubMedBERT_fold4.csv   0.9151481 
sub_sugi_UMLSBert_ENG_fold4.csv           0.9134154
sub_sugi_BiomedNLP-PubMedBERT.csv    0.9129580
sub_sugi_BiomedNLP-PubMedBERT_fold3.csv   0.9114425 
sub_sugi_UMLSBert_ENG.csv                 0.9091279 
sub_sugi_scibert_scivocab_uncased.csv     0.9054163
sub_sugi_bluebert.csv                     0.9051148 
sub_sugi_Bio_Discharge_Summary_BERT.csv   0.9032105 
sub_masa_BioRedditBERT-uncased.csv        0.8991509
sub_sugi_BioRedditBERT-uncased.csv   0.8985702 
sub_sugi_scibert_scivocab_cased.csv       0.8977792 
sub_sugi_COVID-SciBERT.csv                0.8949713
sub_masa_COVID-SciBERT.csv                0.8895940 
```



In [2]:
CSV_LIST = [
               'sub_prob_004',
               'sub_prob_005',
               'sub_prob_006',
               'sub_prob_007',
            #    'sub_sugi_BiomedNLP-PubMedBERT',
            #    'sub_sugi_UMLSBert_ENG',
            #    'sub_sugi_bluebert',
            #    'sub_masa_BioRedditBERT-uncased',
            ]
WEIGHTS = [1,1,1,1,1]

def read_data(csv_name):
    f = '{}/{}.csv'.format(DATA_DIR, csv_name)
    if PROB:
        f = '{}/{}.csv'.format(DATA_DIR, csv_name)

    df_ = pd.read_csv(f, header=None, names=['id', 'prob'])
    # df_[df_.prob >= 0.01].prob.plot.hist(bins=50)
    # plt.show()

    return df_

df = pd.DataFrame()

i = 0
for c in CSV_LIST:
    print(c)
    if len(df) == 0:
        df = read_data(c)
        # print(df.prob.quantile(1-BORDER))
        # df.prob.plot.hist(bins=50)
        df.prob *= WEIGHTS[i]
    else:
        df_ = read_data(c)
        # print(df_.prob.quantile(1-BORDER))
        df_.prob *= WEIGHTS[i]
        df = pd.concat([df, df_['prob']], axis=1)

    i += 1

df.set_index('id', inplace=True)
df

sub_prob_004
sub_prob_005
sub_prob_006
sub_prob_007


,prob,prob,prob,prob
id,,,,
27145,0.000412,0.000517,0.000656,0.000032
27146,0.001937,0.000893,0.000767,0.000150
27147,0.000739,0.000924,0.000792,0.000754
27148,0.000562,0.000874,0.001243,0.000581
27149,0.000794,0.000524,0.000839,0.000736
...,...,...,...,...
67974,0.000909,0.000784,0.000931,0.000985
67975,0.001099,0.000513,0.000645,0.000136
67976,0.001603,0.004436,0.001701,0.000803


In [3]:
df['judgement'] = 0
df['judgement'] = df.sum(axis=1)

if PROB:
    # df['judgement'] = df.judgement / (len(CSV_LIST) * 1.0)
    df['judgement'] = df.judgement / (sum(WEIGHTS) * 1.0)
    df[['judgement']].to_csv('{}/sub_prob_008.csv'.format(DATA_DIR), header=False)
    df['judgement'] = np.where(df.judgement < TH, 0, 1)
else:
    # df['judgement'] = np.where(df.judgement < len(CSV_LIST) // 2, 0, 1)
    df['judgement'] = np.where(df.judgement < sum(WEIGHTS) // 2, 0, 1)

df.reset_index(inplace=True)
print(df.judgement.sum())
df.head(20)

2969


,id,prob,prob,prob,prob,judgement
0,27145,0.000412,0.000517,0.000656,0.000032,0
1,27146,0.001937,0.000893,0.000767,0.000150,0
2,27147,0.000739,0.000924,0.000792,0.000754,0
3,27148,0.000562,0.000874,0.001243,0.000581,0
4,27149,0.000794,0.000524,0.000839,0.000736,0
5,27150,0.000850,0.000678,0.000711,0.000515,0
6,27151,0.007694,0.005596,0.023631,0.044202,0
7,27152,0.001143,0.002413,0.005693,0.001507,0
8,27153,0.001067,0.000797,0.000800,0.001496,0
9,27154,0.003386,0.008998,0.006005,0.011699,0


In [4]:
df[['id', 'judgement']].to_csv('{}/sub_008.csv'.format(DATA_DIR), index=False, header=False)